<a href="https://colab.research.google.com/github/DANU011/PyDataAnalytics/blob/main/WeatherForecastForSeoulCity/notebook/WeatherForecastForSeoulCity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 서울시 평균 기온 예측

## 데이터 확인

### 1. 데이터 읽어오기

In [1]:
import pandas as pd
train = pd.read_csv('train.csv')
submission = pd.read_csv('sample_submission.csv')

display(train.head())
display(submission.head())

,일시,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온
0,1960-01-01,2.2,-5.2,7.4,NaN,68.3,1.7,6.7,NaN,NaN,-1.6
1,1960-01-02,1.2,-5.6,6.8,0.4,87.7,1.3,0.0,NaN,NaN,-1.9
2,1960-01-03,8.7,-2.1,10.8,0.0,81.3,3.0,0.0,NaN,NaN,4.0
3,1960-01-04,10.8,1.2,9.6,0.0,79.7,4.4,2.6,NaN,NaN,7.5
4,1960-01-05,1.3,-8.2,9.5,NaN,44.0,5.1,8.2,NaN,NaN,-4.6


,일시,평균기온
0,2023-01-01,0
1,2023-01-02,0
2,2023-01-03,0
3,2023-01-04,0
4,2023-01-05,0


### 2. train 데이터의 요약 정보를 확인

In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23011 entries, 0 to 23010
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일시      23011 non-null  object 
 1   최고기온    23008 non-null  float64
 2   최저기온    23008 non-null  float64
 3   일교차     23007 non-null  float64
 4   강수량     9150 non-null   float64
 5   평균습도    23011 non-null  float64
 6   평균풍속    23007 non-null  float64
 7   일조합     22893 non-null  float64
 8   일사합     18149 non-null  float64
 9   일조율     22645 non-null  float64
 10  평균기온    23011 non-null  float64
dtypes: float64(10), object(1)
memory usage: 1.9+ MB


### 3. 데이터의 범위 확인

In [3]:
train_min = train['일시'].min()
train_max = train['일시'].max()

sub_min = submission['일시'].min()
sub_max = submission['일시'].max()

display(f"train 데이터 기간 : {train_min} ~ {train_max}")
display(f"test 데이터 기간 : {sub_min} ~ {sub_max}")

'train 데이터 기간 : 1960-01-01 ~ 2022-12-31'

'test 데이터 기간 : 2023-01-01 ~ 2023-12-24'

### 4. 일시 칼럼의 중복 데이터 확인

In [4]:
train[train['일시'].duplicated() == True]

,일시,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온


### 5. 날짜 형식 오류 확인

In [5]:
from datetime import datetime

# `일시`형식이 맞지않는 데이터 확인용
train['일시'] = pd.to_datetime(train['일시'], format='%Y-%m-%d', errors='coerce')
train[train['일시'].isna()]

,일시,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온


### 6. 예상 날짜 범위와 실제 데이터 비교

In [6]:
# 범위에 따른 행의 개수 비교
start_date = datetime(1960, 1, 1)
end_date = datetime(2022, 12, 31)
expected_days = (end_date - start_date).days + 1  # 시작일과 종료일 포함

# 실제 데이터에서 '일시' 컬럼의 고유한 날짜 개수
unique_dates = train['일시'].nunique()
print(f"예상된 날짜 개수: {expected_days}, 실제 고유한 날짜 개수: {unique_dates}")

예상된 날짜 개수: 23011, 실제 고유한 날짜 개수: 23011


### 7. 1년 전 데이터로 평균기온 업데이트

In [7]:
# submission의 `일시` 칼럼을 datetime 자료형으로 변환
submission['일시'] = pd.to_datetime(submission['일시'])

# 1년 전 날짜를 기준으로 평균기온 값 매핑
submission['평균기온'] = submission['일시'].apply(lambda x: train.loc[train['일시'] == x - pd.DateOffset(years=1), '평균기온'].values[0])
submission.head()

,일시,평균기온
0,2023-01-01,-4.3
1,2023-01-02,-1.3
2,2023-01-03,-1.9
3,2023-01-04,-2.5
4,2023-01-05,-2.8


## 시각화